<a href="https://colab.research.google.com/github/wkd-woo/finance-statements/blob/main/%ED%98%84%EA%B8%88%ED%9D%90%EB%A6%84%EC%9D%BC%EA%B4%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U finance-datareader

In [2]:
import pandas as pd
import os
from tqdm import tqdm
import FinanceDataReader as fdr

# 파일별로 분류

In [3]:
import glob

재무상태표 = [] # _01_
손익계산서 = [] # _02_
포괄손익계산서 = [] # _03_
현금흐름표 = [] # _04_
자본변동표 = [] #_05_


for name in glob.glob('/content/drive/MyDrive/22캡스톤디자인/재무일괄/*.txt'):
  if '_01_' in name:
    재무상태표.append(name)
  elif '_02_' in name:
    손익계산서.append(name)
  elif '_03_' in name:
    포괄손익계산서.append(name)
  elif '_04_' in name:
    현금흐름표.append(name)
  elif '_05_' in name:
    자본변동표.append(name)

In [4]:
sample = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

# 데이터 추출

In [5]:
for path in tqdm(현금흐름표):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[52:56] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  df['항목코드'] = df['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # 영업활동현금흐름(ifrs_CashFlowsFromUsedInOperatingActivities)
  영업활동현금흐름 = df[(df['항목코드'] == 'ifrs-full_CashFlowsFromUsedInOperatingActivities') | (df['항목코드'] == 'ifrs_CashFlowsFromUsedInOperatingActivities') | (df['항목명'].str.contains('영업활동'))]
  영업활동현금흐름.rename(columns={당기:"당기"}, inplace=True)
  영업활동현금흐름.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(영업활동현금흐름, sort=False, ignore_index=True)
  

  # 투자활동현금흐름(ifrs_CashFlowsFromUsedInInvestingActivities)
  투자활동현금흐름 = df[(df['항목코드'] == 'ifrs_CashFlowsFromUsedInInvestingActivities') | (df['항목코드'] == 'ifrs-full_CashFlowsFromUsedInInvestingActivities') | (df['항목명'].str.contains('투자활동'))]
  투자활동현금흐름.rename(columns={당기:"당기"}, inplace=True)
  투자활동현금흐름.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(투자활동현금흐름, sort=False, ignore_index=True)

  # 재무활동현금흐름(ifrs_CashFlowsFromUsedInFinancingActivities)
  재무활동현금흐름 = df[(df['항목코드'] == 'ifrs_CashFlowsFromUsedInFinancingActivities') | (df['항목코드'] == 'ifrs-full_CashFlowsFromUsedInFinancingActivities') |(df['항목명'].str.contains('재무활동'))]
  재무활동현금흐름.rename(columns={당기:"당기"}, inplace=True)
  재무활동현금흐름.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(재무활동현금흐름, sort=False, ignore_index=True)


  0%|          | 0/24 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
 21%|██        | 5/24 [00:05<00:21,  1.15s/it]/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 24/24 [00:29<00:00,  1.24s/it]


# 데이터 전처리

In [6]:
sample['종목코드'] = sample['종목코드'].apply(lambda x:x[1:-1])

In [7]:
sample['항목명'] = sample.apply(lambda x: '영업활동현금흐름' if x['항목코드'] == 'ifrs_CashFlowsFromUsedInOperatingActivities' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '영업활동현금흐름' if x['항목코드'] == 'ifrs-full_CashFlowsFromUsedInOperatingActivities' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '투자활동현금흐름' if x['항목코드'] == 'ifrs_CashFlowsFromUsedInInvestingActivities' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '투자활동현금흐름' if x['항목코드'] == 'ifrs-full_CashFlowsFromUsedInInvestingActivities' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '재무활동현금흐름' if x['항목코드'] == 'ifrs_CashFlowsFromUsedInFinancingActivities' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '재무활동현금흐름' if x['항목코드'] == 'ifrs-full_CashFlowsFromUsedInFinancingActivities' else x['항목명'], axis=1)

sample

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2018,1분기보고서,3S,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,103,668,799"
1,060310,2018,1분기보고서,3S,292,dart_AdjustmentsForAssetsLiabilitiesOfOperatin...,영업활동으로 인한 자산부채의 변동,"-239,323,866"
2,095570,2018,1분기보고서,AJ네트웍스,763,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-23,469,830,203"
3,068400,2018,1분기보고서,AJ렌터카,761,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-10,253,127,253"
4,006840,2018,1분기보고서,AK홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"87,092,804,426"
...,...,...,...,...,...,...,...,...
158866,069260,2016,3분기보고서,휴켐스,204,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-35,666,129,320"
158867,010240,2016,3분기보고서,흥국,292,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-7,306,908,074"
158868,189980,2016,3분기보고서,흥국에프엔비,112,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-9,369,683,750"
158869,003280,2016,3분기보고서,흥아해운,501,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"35,038,266,599"


In [8]:
mask = sample['항목명'].str.contains('자산|부채|기타|유출|유입|중단|연결|지급|수취|손실|제거|전|관련|조정|계속')
sample = sample[~mask]

In [9]:
sample['항목명'] = sample.apply(lambda x: '영업활동현금흐름' if 'CashFlowsFromUsedInOperatingActivities' in x['항목코드'] else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '투자활동현금흐름' if 'CashFlowsFromUsedInInvestingActivities' in x['항목코드'] else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '재무활동현금흐름' if 'CashFlowsFromUsedInFinancingActivities' in x['항목코드'] else x['항목명'], axis=1)

sample['항목코드'] = sample.apply(lambda x: 'ifrs_CashFlowsFromUsedInOperatingActivities' if 'CashFlowsFromUsedInOperatingActivities' in x['항목코드'] else x['항목코드'], axis=1)
sample['항목코드'] = sample.apply(lambda x: 'ifrs_CashFlowsFromUsedInInvestingActivities' if 'CashFlowsFromUsedInInvestingActivities' in x['항목코드'] else x['항목코드'], axis=1)
sample['항목코드'] = sample.apply(lambda x: 'ifrs_CashFlowsFromUsedInFinancingActivities' if 'CashFlowsFromUsedInFinancingActivities' in x['항목코드'] else x['항목코드'], axis=1)

sample

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2018,1분기보고서,3S,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,103,668,799"
2,095570,2018,1분기보고서,AJ네트웍스,763,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-23,469,830,203"
3,068400,2018,1분기보고서,AJ렌터카,761,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-10,253,127,253"
4,006840,2018,1분기보고서,AK홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"87,092,804,426"
5,054620,2018,1분기보고서,APS홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,045,549,866"
...,...,...,...,...,...,...,...,...
158866,069260,2016,3분기보고서,휴켐스,204,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-35,666,129,320"
158867,010240,2016,3분기보고서,흥국,292,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-7,306,908,074"
158868,189980,2016,3분기보고서,흥국에프엔비,112,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-9,369,683,750"
158869,003280,2016,3분기보고서,흥아해운,501,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"35,038,266,599"


In [10]:
sample['항목명'] = sample.apply(lambda x: '영업활동현금흐름' if '영업활동' in x['항목명'] else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '투자활동현금흐름' if '투자활동' in x['항목명'] else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '재무활동현금흐름' if '재무활동' in x['항목명'] else x['항목명'], axis=1)

sample['항목코드'] = sample.apply(lambda x: 'ifrs_CashFlowsFromUsedInOperatingActivities' if x['항목명'] == '영업활동현금흐름' else x['항목코드'], axis=1)
sample['항목코드'] = sample.apply(lambda x: 'ifrs_CashFlowsFromUsedInInvestingActivities' if x['항목명'] == '투자활동현금흐름' else x['항목코드'], axis=1)
sample['항목코드'] = sample.apply(lambda x: 'ifrs_CashFlowsFromUsedInFinancingActivities' if x['항목명'] == '재무활동현금흐름' else x['항목코드'], axis=1)

sample

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2018,1분기보고서,3S,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,103,668,799"
2,095570,2018,1분기보고서,AJ네트웍스,763,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-23,469,830,203"
3,068400,2018,1분기보고서,AJ렌터카,761,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-10,253,127,253"
4,006840,2018,1분기보고서,AK홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"87,092,804,426"
5,054620,2018,1분기보고서,APS홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,045,549,866"
...,...,...,...,...,...,...,...,...
158866,069260,2016,3분기보고서,휴켐스,204,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-35,666,129,320"
158867,010240,2016,3분기보고서,흥국,292,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-7,306,908,074"
158868,189980,2016,3분기보고서,흥국에프엔비,112,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-9,369,683,750"
158869,003280,2016,3분기보고서,흥아해운,501,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"35,038,266,599"


In [11]:
sample['항목명'].unique()

array(['영업활동현금흐름', '투자활동현금흐름', '재무활동현금흐름'], dtype=object)

In [12]:
sample['항목코드'].unique()

array(['ifrs_CashFlowsFromUsedInOperatingActivities',
       'ifrs_CashFlowsFromUsedInInvestingActivities',
       'ifrs_CashFlowsFromUsedInFinancingActivities'], dtype=object)

In [13]:
sample['항목명'].unique()

array(['영업활동현금흐름', '투자활동현금흐름', '재무활동현금흐름'], dtype=object)

In [14]:
현금흐름일괄 = sample.drop_duplicates()

# 저장

In [15]:
현금흐름일괄.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/일괄/현금흐름일괄.csv')